In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,moerai,PF,2021-11-13 22:19:05,-22.4333,-151.3333,75.09,73,97,9.71
1,1,chauk,MM,2021-11-13 22:19:06,20.8833,94.8167,77.43,82,100,1.36
2,2,saskylakh,RU,2021-11-13 22:19:06,71.9167,114.0833,-16.91,90,8,7.40
3,3,punta arenas,PH,2021-11-13 22:19:06,8.9897,125.3400,76.12,90,68,4.29
4,4,albany,US,2021-11-13 22:17:04,42.6001,-73.9662,41.38,88,98,1.99


In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,moerai,PF,2021-11-13 22:19:05,-22.4333,-151.3333,75.09,73,97,9.71
1,1,chauk,MM,2021-11-13 22:19:06,20.8833,94.8167,77.43,82,100,1.36
3,3,punta arenas,PH,2021-11-13 22:19:06,8.9897,125.3400,76.12,90,68,4.29
7,7,kalemie,CD,2021-11-13 22:19:07,-5.9475,29.1947,72.77,78,100,8.79
8,8,mitsamiouli,KM,2021-11-13 22:19:07,-11.3847,43.2844,79.83,75,65,11.14
9,9,amboasary,MG,2021-11-13 22:19:08,-25.0333,46.3833,75.04,76,35,3.74
10,10,nsanje,MW,2021-11-13 22:19:08,-16.9200,35.2620,73.80,74,92,2.42
12,12,rikitea,PF,2021-11-13 22:19:08,-23.1203,-134.9692,73.35,73,47,12.53
14,14,new norfolk,LR,2021-11-13 22:19:09,6.6518,-10.5932,72.63,93,97,1.41
21,21,port alfred,CD,2021-11-13 22:19:11,-7.0349,29.7638,77.61,53,88,4.83


In [9]:
preferred_cities_df.count()

City_ID       120
City          120
Country       119
Date          120
Lat           120
Lng           120
Max Temp      120
Humidity      120
Cloudiness    120
Wind Speed    120
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,moerai,PF,75.09,-22.4333,-151.3333,
1,chauk,MM,77.43,20.8833,94.8167,
3,punta arenas,PH,76.12,8.9897,125.3400,
7,kalemie,CD,72.77,-5.9475,29.1947,
8,mitsamiouli,KM,79.83,-11.3847,43.2844,
9,amboasary,MG,75.04,-25.0333,46.3833,
10,nsanje,MW,73.80,-16.9200,35.2620,
12,rikitea,PF,73.35,-23.1203,-134.9692,
14,new norfolk,LR,72.63,6.6518,-10.5932,
21,port alfred,CD,77.61,-7.0349,29.7638,


In [11]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

IndexError: list index out of range

In [ ]:
# Grab the first hotel from the results and store the name.
  try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
  except (IndexError):
    print("Hotel not found... skipping.")

In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,moerai,PF,75.09,-22.4333,-151.3333,Le Manotel
1,chauk,MM,77.43,20.8833,94.8167,Bo Khin Guest House
3,punta arenas,PH,76.12,8.9897,125.3400,Vyjannex boarding house
7,kalemie,CD,72.77,-5.9475,29.1947,Mahobilali
8,mitsamiouli,KM,79.83,-11.3847,43.2844,Foyer ADM
9,amboasary,MG,75.04,-25.0333,46.3833,DISCOVERY AMBOASARY
10,nsanje,MW,73.80,-16.9200,35.2620,Nsanje Discovery Lodge
12,rikitea,PF,73.35,-23.1203,-134.9692,People ThankYou
14,new norfolk,LR,72.63,6.6518,-10.5932,
21,port alfred,CD,77.61,-7.0349,29.7638,Hôtel Kisola


In [1]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

NameError: name 'hotel_df' is not defined

In [16]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [18]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [19]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [2]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_df' is not defined